In [ ]:
!pip install requests
!pip install dbfread
!pip install dbf
!pip install sqlalchemy
!pip install psycopg2

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

import dbf
from dbfread import DBF # Документация https://dbfread.readthedocs.io/en/latest/changes.html

from pandas import DataFrame
import time
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import re
import json

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 200)

# 1 задача. Поиск и запись почтовых индексов в файл dbf (поиск через подключение к внутренней БД)

In [ ]:
def null_string(path_file):
    '''Функция открывает файл, указанный в переменной path_file. В этот файл в колонку POST_IDX (почтовый индекс) 
    производит запись пустой строки'''
    dbf_ls = dbf.Table(path_file)
    dbf_ls.open(dbf.READ_WRITE)

    # loop over records
    for record in dbf_ls:
        with record:
            record['POST_IDX'] = '' #записываем пустую строку 
    
    dbf_ls.close()

In [ ]:
# чтение из БД
def index_from_bd ():
    '''Функция выполняет запрос к базе postgres и возвращает DataFrame с двумя колонками: FIASHOUSE,  POST_IDX'''
        
    df_index = pd.read_sql (
    """
    with house as (  
                    select ah.objectguid, ah.objectid  -- получение кода ГАР дома
                    from as_houses ah  
                    where ah.isactual = 1 and ah.isactive = 1
                    ), 
        params as(
                    select ahp.objectid, ahp.value  
                    from as_houses_params ahp 
                    where ahp.typeid = 5 -- почтовый индекс
                    )
    select house.objectguid as "FIASHOUSE", params.value as "POST_IDX"
    from house
    left join params on house.objectid = params.objectid 

    """,
    "postgresql://pgadmin:1111@ррр/postgres" # вместо 1111 поставить пароль, вместо ррр поставить хост
    )
    return df_index


In [ ]:
# чтение файла DBF
def read_file(path_file):
    global dbf_table
    '''Функция получает на вход dbf файл, открывает его и сохраняет как DataFrame'''
    dbf_table = DBF(path_file)
    dbf_table = DataFrame(iter(dbf_table))
    return dbf_table 

In [ ]:
def join_table(path_file):
    '''Функция получает путь к файлу Dbf, передает этот путь в функцию read_file и получает DataFrame (dbf_table) из данных 
    dbf файла. К dbf_table присодиняется второй DataFrame (df_index), полученный по запросу к БД, по коду ГАР 
    (до уровня дом). Функция возвращет объединенный файл'''
    global df_join
    df_join = read_file(path_file).merge (index_from_bd(), left_on = 'FIASHOUSE', right_on = 'FIASHOUSE', how = 'inner')
    
    return df_join

In [ ]:
def post_from_df_schet(SCHET):
    '''функция берет на вход номер счета из строки df_join. Ищется соответсвие номеру счета в df_join и dbf. 
    Возвращает почтовый индекс конкретного счета'''
    k = df_join.loc[df_join['SCHET'] == SCHET]
    return int (k.iloc[0]['POST_IDX_y'])

In [ ]:
#запись в dbf
def record_file(path_file):
    '''Функция открывает по пути (path_file) файл dbf и производит в него запись почтового индекса в колонку 
    POST_IDX из df_join  '''
    dbf_ls = dbf.Table(path_file)
    dbf_ls.open(dbf.READ_WRITE)

    # loop over records
    for record in dbf_ls:
        with record:
            record['POST_IDX'] = str(post_from_df_schet (record['SCHET'].strip())) #удаляем пробелы из записи со счетом из dbf. Присваиваем найденный почтовый индекс из df
    
    dbf_ls.close()

In [ ]:
def main():
    path_file = input('Введите полный путь к файлу с двумя \\, например C:\\Users\\ПК\\Desktop\\название файла.dbf: ')
    null_string(path_file) #Записываем пустую строку в колонку POST_IDX
    print('В файл dbf записана пустая строка')
    join_table(path_file) # Соединяем исходный файл по коду ФИАС с запросом к БД, получаем новую колонку с индексом
    print('Объединили DataFarme из dbf с запросом SQL, получили почтовые индексы по кодам ГАР до уровня дом')
    record_file(path_file) # Записываем в исходный файл найденный индекс
    print ('Почтовые индексы записаны в файл dbf. Задача выполнена')

In [ ]:
if __name__ == "__main__":
    main()


## Проверка записанных значений

In [ ]:
dbf_table = DBF('') # указать путь к файлу
dbf_table = DataFrame(iter(dbf_table))
dbf_table


In [ ]:
dbf_table [dbf_table['FIAS'].str.contains('^$', regex=True)] # проверка на пустые значения

# 2 задача. Поиск и запись почтовых индексов в файл dbf (поиск через API налоговой)


In [ ]:
# Загрузка dbf в DataFrame
dbf_table = DBF('') # указать путь к файлу
dbf_table = DataFrame(iter(dbf_table))
#Очищаем колонку индекс
dbf_table ['POST_IDX'] = dbf_table ['POST_IDX'].str.replace(r'^\d+', '', regex=True)
dbf_table

In [ ]:
# Если работаем с частью файла, который ранее выгрузили из dbf в Excel
dbf_table = pd.read_excel ('').fillna('') # поставить путь к файлу, сразу удаляем NaN
dbf_table

In [ ]:
for i, row in dbf_table.iloc [0:10000, :].iterrows():
    headers = {
        'accept': 'application/json',
        'master-token': '' # указать мастер-токен
            }

    params = {
        'object_guid': row ['FIASHOUSE'],
        'address_type': '2'
            }

    response = requests.get(
        'https://fias-public-service.nalog.ru/api/spas/v2.0/GetAddressItemByGuid',
        params=params,
        headers=headers
        )
    
    if (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        try:
            json = response.json()
            dbf_table.loc [i,'POST_IDX'] = json['addresses'][0]['address_details']['postal_code'] # почтовый индекс
            
        except JSONDecodeError:
            print ('ошибка JSONDecodeError')
        except ConnectionError:
            print ('разрыв соединения')
    
    
    time.sleep(0.03)  

In [ ]:
# фильтр по обработанным строкам
dbf_table_done = dbf_table [~dbf_table ['POST_IDX'].str.contains('^$', regex=True) ]                              
dbf_table_done.to_excel('1 часть.xlsx', index=False)

# фильтр по НЕ обработанным строкам
dbf_table_not_done = dbf_table [dbf_table ['POST_IDX'].str.contains('^$', regex=True) ]                            
dbf_table_not_done.to_excel('Не выполнено после 1 прогона.xlsx', index=False)

# 3 задача. Работа с Excel файлами на исправление (поиск через API налоговой)

In [ ]:
# чтение файла
df = pd.read_excel ('') # указать путь к файлу
df

In [ ]:
for i, row in dbf_table.iloc [0:10000, :].iterrows():
    headers = {
        'accept': 'application/json',
        'master-token': '' # указать мастер-токен
            }

    params = {
        'object_guid': row ['FIASHOUSE'],
        'address_type': '2'
            }

    response = requests.get(
        'https://fias-public-service.nalog.ru/api/spas/v2.0/GetAddressItemByGuid',
        params=params,
        headers=headers
        )
    
    if (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        try:
            json = response.json()
            dbf_table.loc [i,'POST_IDX_1'] = json['addresses'][0]['address_details']['postal_code'] # почтовый индекс
            
        except JSONDecodeError:
            print ('ошибка JSONDecodeError')
        except ConnectionError:
            print ('разрыв соединения')
    
    
    time.sleep(0.03)  

In [ ]:
#поиск цифровой строки
dbf_table ['POST_IDX'] = dbf_table ['POST_IDX'].str.replace(r'^\d+', '', regex=True)
dbf_table

# 4 задача. Поиск ФИАС (уровень дом), почтового индекса для ООО Водоснабжение (Пласт) 

In [ ]:
# Чтение файла
dbf_table = DBF('') # указать путь к файлу
dbf_table = DataFrame(iter(dbf_table))
dbf_table


In [ ]:
# Проверка на содержание информации в столбце
dbf_table [~dbf_table['HOUSE_LIT'].str.contains('^$', regex=True)]

In [ ]:
for i, row in dbf_table.iloc [0:10000, :].iterrows():
    headers = {
                'accept': 'application/json',
                'master-token': '', #указать мастер-токен
                'Content-Type': 'application/json',
               }

    json_data = {
            'region': { 'name': 'Челябинская область'},
            'district': {'name': 'Пластовский район'},
            'city': { 'name': row['CITY']},
            'street': { 'name': row['STREET']},
            'house': { 'number': str(row['HOUSE'])}
                  }

    response = requests.post('https://fias-public-service.nalog.ru/api/spas/v2.0/SearchByParts', headers=headers, json=json_data)
    
    if (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        try:
            json = response.json()
            print(json)
            if json['error'] is not None:
                dbf_table.loc [i,'FIAS'] = 'Не найден ФИАС до уровня дом' 
                dbf_table.loc [i,'error'] = json['error'] # текст ошибки
                dbf_table.loc [i,'description_inf'] = json['description'] #описание   
            else: 
                dbf_table.loc [i,'FIAS'] = json['address_item']['object_guid'] # код ФИАС до уровня дома 
                dbf_table.loc [i,'full_name_address'] = json['address_item']['full_name'] # полная строка адреса (муниципальное деление)
                dbf_table.loc [i,'POST_IDX'] = json['address_item']['address_details']['postal_code']
        
        except JSONDecodeError:
            print ('ошибка JSONDecodeError')
        except ConnectionError:
            print ('разрыв соединения')
     
    time.sleep(0.02)   

In [ ]:
# Фильтрация и выгрузка данных
dbf_table.to_excel('список без фильтрации.xlsx', index=False)

not_found = dbf_table [dbf_table['FIAS'].str.contains('Не найден ФИАС до уровня дом')]
not_found.to_excel('Не найдены ФИАС.xlsx', index=False)

found = dbf_table [~dbf_table['FIAS'].str.contains('Не найден ФИАС до уровня дом')]
found.to_excel('найдены ФИАС.xlsx', index=False)

In [ ]:
#если работаем с выгрузкой
df = pd.read_excel ('', dtype = str).fillna('') # поставить путь к файлу, сразу удаляем NaN
df['FIAS'] = df['FIAS'].replace ('Не найден ФИАС до уровня дом', '0')
df

In [ ]:
def post_from_df_ls(LS):
    '''функция берет на вход номер ЛС из строки df. Ищется соответсвие номеру счета в df и dbf. 
    Возвращает почтовый индекс конкретного ЛС'''
    k = df.loc[df['LS'] == LS]
    return str (k.iloc[0]['POST_IDX'])

In [ ]:
def post_from_df_fias(LS):
    '''функция берет на вход номер ФИАС из строки df. Ищется соответсвие номеру ФИАС в df и dbf. 
    Возвращает ФИАС конкретного ЛС'''
    l = df.loc[df['LS'] == LS]
    return str(l.iloc[0]['FIAS'])

In [ ]:
dbf_ls = dbf.Table('C:\\Users\\ПК\\Desktop\\LS.DBF')
dbf_ls.open(dbf.READ_WRITE)

    # loop over records
for record in dbf_ls:
    with record:
        record['POST_IDX'] = str(post_from_df_ls (record['LS'].strip())) #удаляем пробелы из записи со счетом из dbf. Присваиваем найденный почтовый индекс из df
        record['FIAS'] = str(post_from_df_fias (record['LS'].strip())) #удаляем пробелы из записи со счетом из dbf. Присваиваем найденный ФИАС из df
dbf_ls.close()


In [ ]:
#Проверка записи
dbf_table = DBF('') # указать путь к файлу
dbf_table = DataFrame(iter(dbf_table))
dbf_table

# 5 задача. Раскрытие домов до уровня квартир для газовой компании. 

In [ ]:
df = pd.read_excel ('', dtype = str).fillna('') # поставить путь к файлу, сразу удаляем NaN
df

In [ ]:
# соединяем дом и литеру
for i, row in df.iterrows():
    if row['Литера'] != '':
        df.loc [i,'house_new'] = str(row['№ дома']) + row['Литера']
    else: df.loc [i,'house_new'] = str(row['№ дома'])

# Составление адресной строки
for i, row in df.iterrows():
    df['address_for_search'] = 'Челябинская область, Челябинск, ' + df ['Адрес'] + ', ' + df ['house_new']
df

In [ ]:
for i, row in df.iloc [0:10000, :].iterrows():
    headers = {
        'accept': 'application/json',
        'master-token': '' # указать мастер-токен
            }

    params = {
            'search_string': row ['address_for_search'],
            'address_type': '2', #Вид представления адреса: 1 - административное деление, 2 - муниципальное деление
             }
    response = requests.get('https://fias-public-service.nalog.ru/api/spas/v2.0/SearchAddressItems', 
                            params=params, headers=headers)
   
    if (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        try:
            json = response.json()
            
             # Записываем первый найденный адрес   
            df.loc [i,'FIAS'] = json['addresses'][0]['object_guid'] # код ГАР 
            df.loc [i,'full_name_address'] = json['addresses'][0]['full_name'] # полная строка адреса (муниципальное деление)
    
           # Записываем второй найденный адрес
            df.loc [i,'FIAS_1'] = json['addresses'][1]['object_guid'] # код ГАР 
            df.loc [i,'full_name_address_1'] = json['addresses'][1]['full_name'] # полная строка адреса (муниципальное деление)
            
        except JSONDecodeError:
            print ('ошибка JSONDecodeError')
        except ConnectionError:
            print ('разрыв соединения')
    
    time.sleep(0.03)  

In [ ]:
#Фильтрация значений
df_filt_done = df[~df['FIAS'].isna()]
df_filt_done.to_excel('find2.xlsx', index=False)

df_filt_done = df[df['FIAS'].isna()]
df_filt_done.to_excel('not_find2.xlsx', index=False)

In [ ]:
#Получаем список ГАР домов
gar_house1 = df_filt_done['FIAS'].iloc[:1000].tolist()
gar_house2 = df_filt_done['FIAS'].iloc[1000:2000].tolist()
gar_house3 = df_filt_done['FIAS'].iloc[2000:3151].tolist()


In [ ]:
print (gar_house1)

In [ ]:
print(gar_house2)

In [ ]:
print (gar_house3)

In [ ]:
df_query_1 = pd.read_sql (
    """
   with rooms 		as (
				select amh.parentobjid, ar.objectid, ar.objectguid, ar.roomtype, ar."number"
				from as_rooms ar 
				left join as_mun_hierarchy amh on amh.objectid = ar.objectid 
				where ar.isactual = 1 and ar.isactive = 1
					),
	param_rooms as (
				-- Параметры комнат
	 			select arp.objectid, arp.typeid, arp.value -- arp.value - кадастровый номер 
	 			from as_rooms_params arp 
	 			where arp.typeid = 8 -- 8- кадастровый номер, 13-Уникальный номер реестровой записи
	 				),	
	flat 		as 	(
					-- данные по квартире
					-- amh.parentobjid - код для связки с домом.
					-- aa.objectguid - ГАР до уровня квартиры
					-- aa."number" - номер квартиры 
					-- aa.aparttype   - тип
					select amh.parentobjid, aa.objectid , aa.objectguid , aa."number" , aa.aparttype  
					from as_apartments aa 
					left join as_adm_hierarchy amh on amh.objectid = aa.objectid 
					where aa.isactual = 1 and aa.isactive = 1					  
					), 
	param_flat as (
	 				-- Данные по кадастровым номерам по квартире
	 				select aap.objectid, aap.typeid, aap.value -- aap.value - кадастровый номер 
	 				from as_apartmens_params aap 
	 				where aap.typeid = 8 -- 8- кадастровый номер, 13-Уникальный номер реестровой записи
	 				),
	house      as (  
					-- данные по дому
					with param_house as (
										-- Параметры дома
										select ahp.objectid, ahp.value  
										from as_houses_params ahp 
										where ahp.typeid = 5 -- почтовый индекс
											and enddate > current_date -- проверка, что индекс действующий
										 )	
					select amh.parentobjid, ah.objectid, ah.objectguid, ah.housenum, param_house.value  -- получение кода ГАР дома
					from as_houses ah  
					left join as_adm_hierarchy amh on amh.objectid = ah.objectid
					left join param_house on ah.objectid = param_house.objectid
					where ah.isactual = 1 and ah.isactive = 1
			  		),
	street    as (
					-- Данные по улице
					select amh.parentobjid as "parentobjid_street", aao.objectid as "id_street",  aao.objectguid as "gar_guid_street", 
					aao.typename as "type_street", aao."name" as "street_name" --aao.objectguid - это ГАР улицы (ФИАС улицы)
					from as_addr_obj aao 
					inner join as_adm_hierarchy amh on amh.objectid = aao.objectid
					where aao.isactual = 1 and 
						  aao.isactive =1 and 
						  aao.enddate > current_date 
			   		),
city as 		(
					-- Населенный пункт
					-- aao.objectid - идентификатор
					-- aao.objectguid - ГАР ГУИД населенного пункта
					-- aao.typename  - тип населенного пункта
					-- aao."name" - наименование населенного пункта
					-- таблицу as_addr_obj_params не присоединяла. В ней ОКТМО и т.д. населенного пункта
					select aadmh.parentobjid, aadmh.regioncode, aao.objectid, aao.objectguid, aao.typename, aao."name" as "city"
					from as_addr_obj aao 
					left join as_adm_hierarchy aadmh on aao.objectid = aadmh.parentobjid
					--left join as_addr_obj_division aaod on aaod.parentid = aao.objectid
					where aao.isactual = 1 and aao.isactive = 1 and aao.enddate > current_date 
					)
select city.typename, city.city, street.type_street, street.street_name, house.value as "post_index", house.objectguid as "gar_guid_house", 
		house.housenum as "house", flat.objectguid as "gar_guid_flat", flat."number" as "flat", flat.aparttype, rooms.objectguid as "gar_guid_room", 
		rooms.roomtype as "type_room", rooms."number" as "room"
from flat 
left join rooms on flat.objectid = rooms.parentobjid
--inner join param_flat on flat.objectid = param_flat.objectid
--inner join param_rooms on rooms.objectid = param_rooms.objectid
right join house on house.objectid = flat.parentobjid
right join street on street.id_street = house.parentobjid 
right join city on city.objectid = street.parentobjid_street
group by city.typename, city.city, street.type_street, street.street_name, house.value, house.objectguid, house.housenum, flat.objectguid, flat."number", 
		flat.aparttype, rooms.objectguid, rooms.roomtype, rooms."number"
having house.objectguid in ('555397f5-ad94-4193-b310-ffc917cf5c37', '78d879a4-04cf-4dbe-b7bc-5bade0147042', 'f2332ab4-9473-433a-abf5-c5136c3d2d7b', '4bbf8d4e-dd39-4ea7-957b-1453b3e40cf2', 'b5fa80ec-3674-46e1-a450-e5a85d7824bd', 'e9849120-a5fc-4db2-b5b7-01d7974caff8', 'c9d8c3fb-f6f6-417b-ae34-6c67f415ee11', 'b76cec3a-28d2-42ad-a632-cd2faeddaeab', 'f240e6e0-b651-4c14-a161-239940c9ccd7', '6d0e23a3-978e-4a1e-825e-284f1200bb5e', '76aacd0d-5721-4624-a715-79f86a7a7164', '37770756-e3b7-402c-8a3b-6b83dac54582', 'e21a6b49-327e-49b4-af60-7ae48b9dc6de', '5f42f1b6-5eef-4ddb-bf99-6b66c1d0e015')

    """,
    "postgresql://pgadmin:1111@ррр/postgres" # вместо 1111 поставить пароль, вместо ррр поставить хост
    )


In [ ]:
# соединяем тип улицы и улицу 
for i, row in df_query_1.iterrows():
    df_query_1.loc [i,'street'] = row['type_street'] + ' ' + row['street_name']
df_query_1

In [ ]:
#Записываем в файл
df_query_1.to_csv('1 часть.csv')

In [ ]:
# соединяем все 3 части
df_query_1 = pd.read_csv('')
df_query_2 = pd.read_csv('')
df_query_3 = pd.read_csv('')

df_union = pd.concat([df_query_1, df_query_2, df_query_3], axis=0)
df_union

In [ ]:
# записыаем объединенный файл в Excel
df_union.to_excel('df_union.xlsx', index=False)

In [ ]:
#Создание dbf
new_table = dbf.Table('gaz_inf3.dbf', 'LS C(10); GIS_ELS C(10); GIS_GKU C(13); GIS_DOC C(18); POST_IDX C(6); CITY C(40); SUBCITY C(30); STREET C(100); HOUSE C(10); HOUSE_LIT C(10); FLAT C(20); LS_SQ N(5, 1); FIO_FULL C(120); FIO C(50); OP_DATE N(5,2); LODGER N(10,5); ZERO C(1); DEBT_MONTH N(10,5); ROOM C(20); DISTRICT C(20); FIAS C(36); EMAIL C(50); STATUS C(1)', codepage='cp866')

In [ ]:
#Записываем в dbf
new_table.open(dbf.READ_WRITE)

for i, row  in df_query_1.iterrows():
    datum = ('', '', '', '', str(row ['post_index']), row ['city'], '', row['street'], row ['house'], '',  row['flat'], 0, '', '', 0, 0, '', 0, '', '', row['gar_guid_house'], '', '')
    
    new_table.append(datum)

new_table.close()       


In [ ]:
dbf_table = DBF('') #поставить путь к файлу
dbf_table = DataFrame(iter(dbf_table))
dbf_table